### Change log

#### 04/28
- Filled missing dates, removed duplicated rows with pivot_table

#### 04/29
- Filled missing data with STMVL method
- there are still data missing, but leave it for now
- normaize the data (not used)

#### 04/30
- use nearest grid to fill meterology data for beijing
- filled missing metero data with STMVL method

#### 05/03
- moved project to new folder

#### 05/05
- finish duplication and missing data analysis

### backlog
- compare results of filling with STMVL and nearest locations (matplotlib)

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
# import
from utils.data_utils import *
from utils.STMVL import STMVL
import pandas as pd
import numpy as np
from data_preprocess import *
import matplotlib.pyplot as plt

### Beijing Air Quality Data

In [3]:
aq, aq_historical, aq_new = load_city_aq('bj')

In [4]:
# process aq_new data
aq_new.drop('id', axis=1, inplace=True)
aq_new.rename(columns={'station_id': 'stationId', 'time': 'utc_time', 'PM25_Concentration': 'PM2.5', 'PM10_Concentration': 'PM10', 'NO2_Concentration': 'NO2', 'CO_Concentration': 'CO', 'O3_Concentration': 'O3', 'SO2_Concentration': 'SO2'}, 
                        inplace=True)

# merge
df_temp = pd.concat([aq, aq_historical, aq_new], ignore_index=True)
df_temp.sort_index(inplace=True)

### Deduplication
There are two types of duplications:
  - Exactly same rows
  - For a station, there are two different values in a same date

#### duplicated rows - exactly same

In [7]:
df_temp.loc[df_temp.duplicated(), :].shape

(6790, 8)

In [8]:
df_temp.loc[df_temp['utc_time'] == '2017-03-02 16:00:00', :]

,stationId,utc_time,PM2.5,PM10,NO2,CO,O3,SO2
1369,aotizhongxin_aq,2017-03-02 16:00:00,76.0,119.0,95.0,1.5,29.0,31.0
1370,aotizhongxin_aq,2017-03-02 16:00:00,76.0,119.0,95.0,1.5,29.0,31.0
10255,badaling_aq,2017-03-02 16:00:00,34.0,87.0,73.0,0.6,2.0,16.0
10256,badaling_aq,2017-03-02 16:00:00,34.0,87.0,73.0,0.6,2.0,16.0
19141,beibuxinqu_aq,2017-03-02 16:00:00,97.0,214.0,71.0,1.8,2.0,9.0
19142,beibuxinqu_aq,2017-03-02 16:00:00,97.0,214.0,71.0,1.8,2.0,9.0
28027,daxing_aq,2017-03-02 16:00:00,68.0,131.0,43.0,0.7,34.0,25.0
28028,daxing_aq,2017-03-02 16:00:00,68.0,131.0,43.0,0.7,34.0,25.0
36913,dingling_aq,2017-03-02 16:00:00,79.0,86.0,41.0,1.9,44.0,18.0
36914,dingling_aq,2017-03-02 16:00:00,79.0,86.0,41.0,1.9,44.0,18.0


2017-03-02 16:00:00 has duplication. We validate they are removed after conversion.

In [14]:
# aq_city_preprocess('bj')
data_filled = aq_city_preprocess('bj')
# data_filled = pd.read_csv('./data/Beijing/post/bj_PM2.5_filled.csv', index_col=0)

start working on bj air quality


In [15]:
data_filled.loc[data_filled.duplicated(), :].shape

(10, 35)

After convert the data from long format to wide format, it automatically removed duplicated rows.

In [16]:
data_filled.loc[data_filled.index == '2017-03-02 16:00:00', :]

,aotizhongxin_aq_PM2.5,badaling_aq_PM2.5,beibuxinqu_aq_PM2.5,daxing_aq_PM2.5,dingling_aq_PM2.5,donggaocun_aq_PM2.5,dongsi_aq_PM2.5,dongsihuan_aq_PM2.5,fangshan_aq_PM2.5,fengtaihuayuan_aq_PM2.5,...,wanliu_aq_PM2.5,wanshouxigong_aq_PM2.5,xizhimenbei_aq_PM2.5,yanqin_aq_PM2.5,yizhuang_aq_PM2.5,yongdingmennei_aq_PM2.5,yongledian_aq_PM2.5,yufa_aq_PM2.5,yungang_aq_PM2.5,zhiwuyuan_aq_PM2.5
utc_time,,,,,,,,,,,,,,,,,,,,,
2017-03-02 16:00:00,76.0,34.0,97.0,68.0,79.0,53.0,80.0,79.0,80.0,101.0,...,71.0,83.0,82.0,61.0,76.0,83.0,45.0,55.0,73.0,58.0


Values are filled correctly and no duplication left.

#### duplicated time but with different values

Let's check if there's record where dates are same but values are different.

In [17]:
def find_dup_time(group):
    return group.loc[group.index.duplicated(), :].shape

In [18]:
df_temp.groupby('stationId').apply(find_dup_time)

stationId
aotizhongxin_aq      (0, 8)
badaling_aq          (0, 8)
beibuxinqu_aq        (0, 8)
daxing_aq            (0, 8)
dingling_aq          (0, 8)
donggaocun_aq        (0, 8)
dongsi_aq            (0, 8)
dongsihuan_aq        (0, 8)
fangshan_aq          (0, 8)
fengtaihuayuan_aq    (0, 8)
guanyuan_aq          (0, 8)
gucheng_aq           (0, 8)
huairou_aq           (0, 8)
liulihe_aq           (0, 8)
mentougou_aq         (0, 8)
miyun_aq             (0, 8)
miyunshuiku_aq       (0, 8)
nansanhuan_aq        (0, 8)
nongzhanguan_aq      (0, 8)
pingchang_aq         (0, 8)
pinggu_aq            (0, 8)
qianmen_aq           (0, 8)
shunyi_aq            (0, 8)
tiantan_aq           (0, 8)
tongzhou_aq          (0, 8)
wanliu_aq            (0, 8)
wanshouxigong_aq     (0, 8)
xizhimenbei_aq       (0, 8)
yanqin_aq            (0, 8)
yizhuang_aq          (0, 8)
yongdingmennei_aq    (0, 8)
yongledian_aq        (0, 8)
yufa_aq              (0, 8)
yungang_aq           (0, 8)
zhiwuyuan_aq         (0, 8)
dtype: obj

There's no such case in the data, therefore, __pivot_table() can handle all duplication cases__

In [19]:
data_filled.tail()

,aotizhongxin_aq_PM2.5,badaling_aq_PM2.5,beibuxinqu_aq_PM2.5,daxing_aq_PM2.5,dingling_aq_PM2.5,donggaocun_aq_PM2.5,dongsi_aq_PM2.5,dongsihuan_aq_PM2.5,fangshan_aq_PM2.5,fengtaihuayuan_aq_PM2.5,...,wanliu_aq_PM2.5,wanshouxigong_aq_PM2.5,xizhimenbei_aq_PM2.5,yanqin_aq_PM2.5,yizhuang_aq_PM2.5,yongdingmennei_aq_PM2.5,yongledian_aq_PM2.5,yufa_aq_PM2.5,yungang_aq_PM2.5,zhiwuyuan_aq_PM2.5
utc_time,,,,,,,,,,,,,,,,,,,,,
2018-05-03 15:00:00,23.0,43.0,20.0,29.0,NaN,14.0,31.0,27.0,27.0,19.0,...,23.0,20.0,41.0,31.0,26.0,20.0,25.0,31.0,19.0,NaN
2018-05-03 16:00:00,26.0,29.0,20.0,35.0,NaN,12.0,29.0,27.0,28.0,38.0,...,25.0,30.0,41.0,33.0,26.0,26.0,27.0,24.0,26.0,NaN
2018-05-03 17:00:00,26.0,26.0,20.0,32.0,NaN,27.0,30.0,40.0,25.0,41.0,...,25.0,28.0,121.0,28.0,32.0,29.0,33.0,28.0,30.0,NaN
2018-05-03 18:00:00,29.0,NaN,14.0,33.0,NaN,25.0,32.0,39.0,29.0,42.0,...,28.0,28.0,54.0,20.0,28.0,35.0,29.0,26.0,32.0,NaN
2018-05-03 19:00:00,27.0,36.0,17.0,46.0,NaN,19.0,37.0,42.0,35.0,37.0,...,23.0,29.0,52.0,21.0,36.0,40.0,47.0,26.0,28.0,NaN


Investigated into the zhiwuyuan_aq case, it is consistent with original data

In [20]:
df_raw = pd.read_csv('./data/Beijing/aq/bj_aq_new.csv')

In [21]:
df_temp.loc[df_temp['stationId'] == 'zhiwuyuan_aq', :].tail()

,stationId,utc_time,PM2.5,PM10,NO2,CO,O3,SO2
386303,zhiwuyuan_aq,2018-05-03 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN
386338,zhiwuyuan_aq,2018-05-03 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN
386373,zhiwuyuan_aq,2018-05-03 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN
386408,zhiwuyuan_aq,2018-05-03 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN
386443,zhiwuyuan_aq,2018-05-03 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN


### Missing data
There are 3 types of missing data
  - For all staions, all air quality data are missing, including:
      - Some dates are not in the datasset
      - Dates exist with value NaN
  - For one stations, all air quality data are missing
  - For on stations, a few air quality data are missing
  
We use STMVL method to fill the missing data first and validate the results

#### Load data

In [22]:
bj_particles_aq = {}

for particle in bj_particles:
    file_name = 'bj_' + particle + '_filled.csv'
    bj_particles_aq[particle] = pd.read_csv('./Data/Beijing/post/{}'.format(file_name), index_col=0)

#### 1.a find missing dates and fill them

In [23]:
def find_missing_hours(df):
    start_time = df.index.min()
    end_time = df.index.max()
    all_times = pd.date_range(start_time, end_time, freq='H')
    print('There should be {} dates in total'.format(len(all_times)))
    missing_time = np.setdiff1d(all_times.values, df.index.values.astype(np.datetime64))
    print('There are {} missing hours in the data'.format(len(missing_time)))

In [24]:
for particle, data in bj_particles_aq.items():
    print(particle)
    find_missing_hours(data)

PM2.5
There should be 11694 dates in total
There are 0 missing hours in the data
PM10
There should be 11694 dates in total
There are 0 missing hours in the data
NO2
There should be 11694 dates in total
There are 0 missing hours in the data
CO
There should be 11694 dates in total
There are 0 missing hours in the data
O3
There should be 11694 dates in total
There are 0 missing hours in the data
SO2
There should be 11694 dates in total
There are 0 missing hours in the data


There is 846 missing hours in the data

In [12]:
for particle, data in bj_particles_aq.items():
    print(particle)
    bj_particles_aq[particle] = fill_missing_time_with_na(data)

PM2.5
There should be 11694 date time in total
After filling, there are 11694 date time in total
PM10
There should be 11694 date time in total
After filling, there are 11694 date time in total
NO2
There should be 11694 date time in total
After filling, there are 11694 date time in total
CO
There should be 11694 date time in total
After filling, there are 11694 date time in total
O3
There should be 11694 date time in total
After filling, there are 11694 date time in total
SO2
There should be 11694 date time in total
After filling, there are 11694 date time in total


#### 1.b find dates where all features are missing

In [13]:
def find_all_missing(df):
    rows = df.loc[df.isnull().all(axis=1)].shape[0]
    print('There are {} of rows are missing'.format(rows))

In [14]:
for particle, data in bj_particles_aq.items():
    print(particle)
    find_all_missing(data)

PM2.5
There are 857 of rows are missing
PM10
There are 958 of rows are missing
NO2
There are 858 of rows are missing
CO
There are 1579 of rows are missing
O3
There are 857 of rows are missing
SO2
There are 857 of rows are missing


Together with other missing data, we use STMVL to fill them

Validate how STMVL fills the missing data, compared with nearest station.

In [33]:
# find (index, col) of missing values first
pm25 =bj_particles_aq['PM2.5']
is_missing = []
for index in pm25.index:
    row = pm25.loc[index]
    # print(row)
    for col in row.index:
        if np.isnan(row[col]):
            is_missing.append((index, col))
# is_missing = pm25.isnull().any(axis=1)

In [35]:
len(is_missing)

53834

In [36]:
# get nearest lists of stations 
bj_stations = get_stations('bj')
nearest = find_nearest(bj_stations, bj_stations)
len(nearest)

35

In [37]:
# fill NaN values with values from nearest stations
for i in range(len(is_missing)):
    missing_idx = is_missing[i]
    pm25_copy = pm25.copy(deep=True)
    missing_station, missing_feature = missing_idx[1].rsplit('_', maxsplit=1)
    nearest_station = nearest[missing_station]
    nearest_col = nearest_station + '_' + missing_feature
    
    nearest_value = pm25_copy.loc[missing_idx[0], nearest_col]
    if i % 5000 == 0:
        print('Filling missing data for ({}, {})'.format(missing_idx[0], missing_idx[1]))
        print('Filling with {} of ({}, {})'.format(nearest_value, missing_idx[0], nearest_col))
    pm25_copy.loc[missing_idx] = nearest_value

pm25_copy.isnull().any(axis=1).sum()

Filling missing data for (2017-01-01 18:00:00, nongzhanguan_aq_PM2.5)
Filling with 408.0 of (2017-01-01 18:00:00, dongsihuan_aq_PM2.5)
Filling missing data for (2017-03-09 23:00:00, shunyi_aq_PM2.5)
Filling with nan of (2017-03-09 23:00:00, huairou_aq_PM2.5)
Filling missing data for (2017-04-20 15:00:00, guanyuan_aq_PM2.5)
Filling with nan of (2017-04-20 15:00:00, xizhimenbei_aq_PM2.5)
Filling missing data for (2017-05-20 23:00:00, badaling_aq_PM2.5)
Filling with nan of (2017-05-20 23:00:00, yanqin_aq_PM2.5)
Filling missing data for (2017-05-26 21:00:00, yongledian_aq_PM2.5)
Filling with nan of (2017-05-26 21:00:00, tongzhou_aq_PM2.5)
Filling missing data for (2017-06-12 12:00:00, nansanhuan_aq_PM2.5)
Filling with 37.0 of (2017-06-12 12:00:00, wanshouxigong_aq_PM2.5)
Filling missing data for (2017-07-05 11:00:00, guanyuan_aq_PM2.5)
Filling with nan of (2017-07-05 11:00:00, xizhimenbei_aq_PM2.5)
Filling missing data for (2017-07-23 00:00:00, wanliu_aq_PM2.5)
Filling with nan of (2017-07

10295

In [1]:
def plot_ts(feature, feature_after, feature_before):
    
    fig = plt.figure(figsize=(6,4))
    ax = fig.add_subplot(111)
    fig.suptitle('Filling with nearest values:' + feature)
    
#     x = range(df.shape[0])
#     x_fill = [i for i in x if missing_idx[i]]
    # print(x_fill)
    feature_before_ = feature_before.loc[feature_before.isnull()]
    ax.plot(feature_after)
    # ax.scatter(feature_before_, c='r', marker='.', s=0.7)
    # ax.set_xticklabels(df['utc_time'])

In [16]:
ex_feature = pm25_copy.columns[0]
example = pm25_copy.iloc[:, 0]
example_before = pm25.iloc[:, 0]
example.plot()

NameError: name 'pm25_copy' is not defined

In [ ]:
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
fig.suptitle('Filling with nearest values:' + ex_feature)
ax.plot(example)

In [1]:
plt.show()

NameError: name 'plt' is not defined

In [ ]:
ex_feature = pm25_copy.columns[0]
example = pm25_copy.iloc[:, 0]
example_before = pm25.iloc[:, 0]
plot_ts(ex_feature, example, example_before)

In [41]:
is_all_missing = pm25.isnull().all(axis=1)
pm25_copy.loc[is_all_missing, :]

,aotizhongxin_aq_PM2.5,badaling_aq_PM2.5,beibuxinqu_aq_PM2.5,daxing_aq_PM2.5,dingling_aq_PM2.5,donggaocun_aq_PM2.5,dongsi_aq_PM2.5,dongsihuan_aq_PM2.5,fangshan_aq_PM2.5,fengtaihuayuan_aq_PM2.5,...,wanliu_aq_PM2.5,wanshouxigong_aq_PM2.5,xizhimenbei_aq_PM2.5,yanqin_aq_PM2.5,yizhuang_aq_PM2.5,yongdingmennei_aq_PM2.5,yongledian_aq_PM2.5,yufa_aq_PM2.5,yungang_aq_PM2.5,zhiwuyuan_aq_PM2.5
2017-01-23 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-23 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-24 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-24 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-24 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-24 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-24 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-24 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-24 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-25 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
pm25_copy.tail()

,aotizhongxin_aq_PM2.5,badaling_aq_PM2.5,beibuxinqu_aq_PM2.5,daxing_aq_PM2.5,dingling_aq_PM2.5,donggaocun_aq_PM2.5,dongsi_aq_PM2.5,dongsihuan_aq_PM2.5,fangshan_aq_PM2.5,fengtaihuayuan_aq_PM2.5,...,wanliu_aq_PM2.5,wanshouxigong_aq_PM2.5,xizhimenbei_aq_PM2.5,yanqin_aq_PM2.5,yizhuang_aq_PM2.5,yongdingmennei_aq_PM2.5,yongledian_aq_PM2.5,yufa_aq_PM2.5,yungang_aq_PM2.5,zhiwuyuan_aq_PM2.5
2018-05-03 15:00:00,23.0,43.0,20.0,29.0,NaN,14.0,31.0,27.0,27.0,19.0,...,23.0,20.0,41.0,31.0,26.0,20.0,25.0,31.0,19.0,NaN
2018-05-03 16:00:00,26.0,29.0,20.0,35.0,NaN,12.0,29.0,27.0,28.0,38.0,...,25.0,30.0,41.0,33.0,26.0,26.0,27.0,24.0,26.0,NaN
2018-05-03 17:00:00,26.0,26.0,20.0,32.0,NaN,27.0,30.0,40.0,25.0,41.0,...,25.0,28.0,121.0,28.0,32.0,29.0,33.0,28.0,30.0,NaN
2018-05-03 18:00:00,29.0,NaN,14.0,33.0,NaN,25.0,32.0,39.0,29.0,42.0,...,28.0,28.0,54.0,20.0,28.0,35.0,29.0,26.0,32.0,NaN
2018-05-03 19:00:00,27.0,36.0,17.0,46.0,NaN,19.0,37.0,42.0,35.0,37.0,...,23.0,29.0,52.0,21.0,36.0,40.0,47.0,26.0,28.0,23.0


In [46]:
len(is_all_missing)

11694

In [54]:
aq_city_preprocess('bj')

start working on bj air quality
There should be 11694 date time in total
After filling, there are 11694 date time in total
There are 10295 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 465 still missing in the data 

There should be 11694 date time in total
After filling, there are 11694 date time in total
There are 11469 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 482 still missing in the data 

There should be 11694 date time in total
After filling, there are 11694 date time in total
There are 10010 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 465 still missing in the data 

There should be 11694 date time in total
After filling, there are 11694 date time in total
There are 10398 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 1016 still missing in the data 

There should be 11694 date time in total
After filling, there are 11694 date time in total
There are 10

In [140]:
bj_aq = pd.read_csv(r'./Data/Beijing/beijing_17_18_aq.csv')
bj_aq_201802_201803 = pd.read_csv(r'./Data/Beijing/beijing_201802_201803_aq.csv')
bj_aq_new = pd.read_csv(r'./Data/Beijing/bj_aq_new.csv')

In [78]:
bj_particles_aq = load_city_aq('bj')

In [80]:
bj_particles_aq['PM2.5'].tail()

,aotizhongxin_aq_PM2.5,badaling_aq_PM2.5,beibuxinqu_aq_PM2.5,daxing_aq_PM2.5,dingling_aq_PM2.5,donggaocun_aq_PM2.5,dongsi_aq_PM2.5,dongsihuan_aq_PM2.5,fangshan_aq_PM2.5,fengtaihuayuan_aq_PM2.5,...,wanliu_aq_PM2.5,wanshouxigong_aq_PM2.5,xizhimenbei_aq_PM2.5,yanqin_aq_PM2.5,yizhuang_aq_PM2.5,yongdingmennei_aq_PM2.5,yongledian_aq_PM2.5,yufa_aq_PM2.5,yungang_aq_PM2.5,zhiwuyuan_aq_PM2.5
utc_time,,,,,,,,,,,,,,,,,,,,,
2018-05-02 18:00:00,12.0,15.0,9.0,NaN,11.0,5.0,14.0,13.0,14.0,17.0,...,9.0,3.0,25.0,NaN,6.0,6.0,25.0,12.0,7.0,NaN
2018-05-02 19:00:00,11.0,15.0,12.0,NaN,NaN,3.0,13.0,18.0,10.0,11.0,...,11.0,18.0,20.0,25.0,11.0,8.0,24.0,23.0,5.0,NaN
2018-05-02 21:00:00,9.0,21.0,11.0,NaN,5.0,3.0,18.0,10.0,3.0,18.0,...,11.0,14.0,28.0,22.0,14.0,5.0,29.0,19.0,7.0,NaN
2018-05-02 22:00:00,11.0,22.0,11.0,NaN,4.0,3.0,11.0,7.0,9.0,18.0,...,8.0,15.0,23.0,32.0,12.0,9.0,23.0,19.0,8.0,NaN
2018-05-03 00:00:00,25.0,20.0,19.0,NaN,4.0,3.0,19.0,8.0,12.0,25.0,...,8.0,10.0,12.0,37.0,15.0,16.0,15.0,33.0,11.0,NaN


In [134]:
bj_aq_201802_201803.loc[bj_aq_201802_201803['stationId'] == 'dongsi_aq', :].tail(20)

,stationId,utc_time,PM2.5,PM10,NO2,CO,O3,SO2
28568,dongsi_aq,2018-03-30 20:00:00,97.0,169.0,117.0,1.0,2.0,17.0
28569,dongsi_aq,2018-03-30 21:00:00,96.0,179.0,101.0,1.2,2.0,16.0
28570,dongsi_aq,2018-03-30 22:00:00,98.0,165.0,107.0,1.2,2.0,17.0
28571,dongsi_aq,2018-03-30 23:00:00,101.0,163.0,108.0,1.2,2.0,17.0
28572,dongsi_aq,2018-03-31 00:00:00,100.0,157.0,104.0,1.2,4.0,17.0
28573,dongsi_aq,2018-03-31 01:00:00,108.0,162.0,94.0,1.3,18.0,26.0
28574,dongsi_aq,2018-03-31 02:00:00,103.0,141.0,80.0,1.2,45.0,28.0
28575,dongsi_aq,2018-03-31 03:00:00,104.0,140.0,77.0,1.1,57.0,24.0
28576,dongsi_aq,2018-03-31 04:00:00,99.0,127.0,67.0,1.0,80.0,19.0
28577,dongsi_aq,2018-03-31 05:00:00,109.0,139.0,63.0,1.0,97.0,18.0


In [136]:
bj_aq_201802_201803.tail()

,stationId,utc_time,PM2.5,PM10,NO2,CO,O3,SO2
49415,zhiwuyuan_aq,2018-03-31 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN
49416,zhiwuyuan_aq,2018-03-31 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
49417,zhiwuyuan_aq,2018-03-31 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN
49418,zhiwuyuan_aq,2018-03-31 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN
49419,zhiwuyuan_aq,2018-03-31 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
bj_aq_new.head()

,satationId,utc_time,PM2.5,PM10,NO2,CO,O3,SO2
0,dongsi_aq,2018-03-31 07:00:00,105.0,172.0,53.0,0.8,127.0,14.0
1,tiantan_aq,2018-03-31 07:00:00,95.0,123.0,54.0,0.9,121.0,15.0
2,guanyuan_aq,2018-03-31 07:00:00,95.0,139.0,66.0,0.8,123.0,13.0
3,wanshouxigong_aq,2018-03-31 07:00:00,99.0,172.0,58.0,1.0,119.0,14.0
4,aotizhongxin_aq,2018-03-31 07:00:00,117.0,166.0,67.0,0.8,111.0,14.0


In [4]:
particles_aq = load_city_aq('bj')

#### Take PM2.5 as an example

In [5]:
pm25 = particles_aq['PM2.5']

In [6]:
print("start date：", pm25.index.min())
print("end_date：", pm25.index.max())

start date： 2017-01-01 14:00:00
end_date： 2018-04-28 13:00:00


In [7]:
pm25.loc[pm25.index.duplicated(), :]

,aotizhongxin_aq_PM2.5,badaling_aq_PM2.5,beibuxinqu_aq_PM2.5,daxing_aq_PM2.5,dingling_aq_PM2.5,donggaocun_aq_PM2.5,dongsi_aq_PM2.5,dongsihuan_aq_PM2.5,fangshan_aq_PM2.5,fengtaihuayuan_aq_PM2.5,...,wanliu_aq_PM2.5,wanshouxigong_aq_PM2.5,xizhimenbei_aq_PM2.5,yanqin_aq_PM2.5,yizhuang_aq_PM2.5,yongdingmennei_aq_PM2.5,yongledian_aq_PM2.5,yufa_aq_PM2.5,yungang_aq_PM2.5,zhiwuyuan_aq_PM2.5
utc_time,,,,,,,,,,,,,,,,,,,,,


Using `pandas.Dataframe.pivot_table`, it automatically drop duplicated rows by taking average of duplicated rows.

### Does data cover all dates

In [8]:
start_time = pm25.index.min()
end_time = pm25.index.max()
all_time = pd.date_range(start_time, end_time, freq='H')
print('There should be {} dates in total'.format(len(all_time)))

There should be 11568 dates in total


In [9]:
len(pm25.index)

10733

In [10]:
pm25_fill = fill_missing_time_with_na(pm25)

In [11]:
pm25_fill.shape

(11568, 35)

In [12]:
bj_aq.loc[bj_aq['utc_time'] == '2017-01-23 15:00:00', :]

,stationId,utc_time,PM2.5,PM10,NO2,CO,O3,SO2


In [ ]:
pm25_fill.to_csv(r'./Data/')

### Missing data preprocess

In [13]:
missing_idx = bj_aq.loc[:, 'PM2.5':'SO2'].isnull().all(axis=1)
bj_aq_missing = bj_aq.loc[missing_idx, :]

In [20]:
pm25_fill.loc[pm25_fill.isnull().all(axis=1), :].shape

(846, 35)

In [15]:
pm25_filled = fill_missing_single_data(pm25_fill, stations)

STMVL: Preparing
STMVL: Start with 8 processes


In [17]:
pm25_filled.loc[pm25_filled.isnull().any(axis=1), :].shape

(465, 35)

In [19]:
np.isnan(pm25_filled).any(axis=1).sum()

465

After using STMVL to fill the missing data, there are 439 data points left as missing

In [21]:
pm25_filled.to_csv(r'./Data/pm25_filled.csv', index=True)

In [25]:
still_missing = pm25_filled.loc[pm25_filled.isnull().any(axis=1), :]
still_missing.index.tolist()

['2017-01-26 13:00:00',
 '2017-01-26 14:00:00',
 '2017-01-26 15:00:00',
 '2017-01-26 16:00:00',
 '2017-03-13 13:00:00',
 '2017-03-13 14:00:00',
 '2017-03-13 15:00:00',
 '2017-03-13 16:00:00',
 '2017-03-13 17:00:00',
 '2017-03-13 18:00:00',
 '2017-04-23 12:00:00',
 '2017-04-23 13:00:00',
 '2017-04-23 14:00:00',
 '2017-04-23 15:00:00',
 '2017-04-23 16:00:00',
 '2017-04-23 17:00:00',
 '2017-04-23 18:00:00',
 '2017-04-23 19:00:00',
 '2017-05-18 12:00:00',
 '2017-05-18 13:00:00',
 '2017-05-18 14:00:00',
 '2017-05-18 15:00:00',
 '2017-05-18 16:00:00',
 '2017-05-18 17:00:00',
 '2017-05-18 18:00:00',
 '2017-05-18 19:00:00',
 '2017-05-18 20:00:00',
 '2017-05-18 21:00:00',
 '2017-05-18 22:00:00',
 '2017-05-18 23:00:00',
 '2017-05-19 00:00:00',
 '2017-05-19 01:00:00',
 '2017-05-19 02:00:00',
 '2017-05-19 03:00:00',
 '2017-05-19 04:00:00',
 '2017-05-19 05:00:00',
 '2017-05-19 06:00:00',
 '2017-05-19 07:00:00',
 '2017-05-19 08:00:00',
 '2017-05-19 09:00:00',
 '2017-05-19 10:00:00',
 '2017-05-19 11:

#### Using backward fill to fill the left missing data (__Not used yet__)

In [32]:
temp = pm25_filled.fillna(method='bfill', axis=0)

####  Run preprocess for all particles

In [40]:
aq_city_preprocess('bj')

Fill missing data for PM2.5
There are 10169 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 465 still missing in the data 

Fill missing data for PM10
There are 11343 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 482 still missing in the data 

Fill missing data for NO2
There are 9884 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 465 still missing in the data 

Fill missing data for CO
There are 10272 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 1016 still missing in the data 

Fill missing data for O3
There are 10119 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 472 still missing in the data 

Fill missing data for SO2
There are 9861 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 465 still missing in the data 



### Normalization

In [57]:
for particle in particles_aq.keys():
    print(particle)
    print('======================================')
    data_filled = pd.read_csv(r'./Data/{}_filled.csv'.format(particle), index_col=0)
    # print(data_filled.describe())
    print('number of negative values: {}'.format((data_filled < 0).sum()))
    print('======================================')

PM2.5
number of negative values: aotizhongxin_aq_PM2.5      0
badaling_aq_PM2.5          0
beibuxinqu_aq_PM2.5        0
daxing_aq_PM2.5            0
dingling_aq_PM2.5          0
donggaocun_aq_PM2.5        1
dongsi_aq_PM2.5            0
dongsihuan_aq_PM2.5        0
fangshan_aq_PM2.5          0
fengtaihuayuan_aq_PM2.5    0
guanyuan_aq_PM2.5          0
gucheng_aq_PM2.5           0
huairou_aq_PM2.5           0
liulihe_aq_PM2.5           0
mentougou_aq_PM2.5         0
miyun_aq_PM2.5             0
miyunshuiku_aq_PM2.5       0
nansanhuan_aq_PM2.5        0
nongzhanguan_aq_PM2.5      0
pingchang_aq_PM2.5         0
pinggu_aq_PM2.5            0
qianmen_aq_PM2.5           0
shunyi_aq_PM2.5            0
tiantan_aq_PM2.5           0
tongzhou_aq_PM2.5          0
wanliu_aq_PM2.5            0
wanshouxigong_aq_PM2.5     0
xizhimenbei_aq_PM2.5       0
yanqin_aq_PM2.5            0
yizhuang_aq_PM2.5          0
yongdingmennei_aq_PM2.5    0
yongledian_aq_PM2.5        0
yufa_aq_PM2.5              0
yungang_aq

There are some negative values in the dataset, which I don't know how to deal since it can be low concentration and near detection limit, etc. See https://www.researchgate.net/post/How_can_I_deal_with_negative_and_zero_concentrations_of_PM25_PM10_and_gas_data

Since test data will also have this calibration issue, I will not deal with it now. (__can convert it to positive later__)

In [266]:
station = 'zhiwuyuan_aq'
bj_aq_missing.loc[bj_aq_missing['stationId'] == station, :].shape

(2067, 8)

In [38]:
bj_aq.tail()

,stationId,utc_time,PM2.5,PM10,NO2,CO,O3,SO2
311005,zhiwuyuan_aq,2018-01-31 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN
311006,zhiwuyuan_aq,2018-01-31 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
311007,zhiwuyuan_aq,2018-01-31 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN
311008,zhiwuyuan_aq,2018-01-31 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN
311009,zhiwuyuan_aq,2018-01-31 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
bj_aq_201802_201803.head()

,stationId,utc_time,PM2.5,PM10,NO2,CO,O3,SO2
0,aotizhongxin_aq,2018-01-31 16:00:00,49.0,82.0,90.0,0.9,6.0,10.0
1,aotizhongxin_aq,2018-01-31 17:00:00,47.0,80.0,90.0,0.9,5.0,10.0
2,aotizhongxin_aq,2018-01-31 18:00:00,46.0,91.0,91.0,1.3,5.0,28.0
3,aotizhongxin_aq,2018-01-31 19:00:00,60.0,95.0,85.0,2.0,6.0,38.0
4,aotizhongxin_aq,2018-01-31 20:00:00,52.0,91.0,81.0,1.9,5.0,30.0


In [36]:
bj_aq_201802_201803.tail(800)

,stationId,utc_time,PM2.5,PM10,NO2,CO,O3,SO2
48620,yungang_aq,2018-03-29 08:00:00,33.0,157.0,14.0,0.4,122.0,7.0
48621,yungang_aq,2018-03-29 09:00:00,30.0,167.0,18.0,0.3,118.0,5.0
48622,yungang_aq,2018-03-29 10:00:00,31.0,180.0,20.0,0.4,114.0,4.0
48623,yungang_aq,2018-03-29 11:00:00,27.0,256.0,22.0,0.4,98.0,5.0
48624,yungang_aq,2018-03-29 12:00:00,34.0,178.0,29.0,0.6,79.0,7.0
48625,yungang_aq,2018-03-29 13:00:00,27.0,125.0,27.0,0.5,75.0,7.0
48626,yungang_aq,2018-03-29 14:00:00,25.0,91.0,28.0,0.4,71.0,6.0
48627,yungang_aq,2018-03-29 15:00:00,22.0,98.0,47.0,0.5,42.0,5.0
48628,yungang_aq,2018-03-29 16:00:00,27.0,95.0,47.0,0.5,39.0,5.0
48629,yungang_aq,2018-03-29 17:00:00,26.0,88.0,46.0,0.6,35.0,5.0


### Beijing Grid Metero Data

In [68]:
bj_meter_grid_hist = pd.read_csv(file_paths['bj_meter_grid_hist'])
bj_meter_grid_new = pd.read_csv(file_paths['bj_meter_grid_new'])

In [72]:
metero_stations = load_city_meter('bj')

temperature
pressure
humidity
wind_direction
wind_speed


In [69]:
bj_meter_grid_hist.head()

,stationName,longitude,latitude,utc_time,temperature,pressure,humidity,wind_direction,wind_speed/kph
0,beijing_grid_000,115.0,39.0,2017-01-01 00:00:00,-5.47,984.73,76.60,53.71,3.53
1,beijing_grid_001,115.0,39.1,2017-01-01 00:00:00,-5.53,979.33,75.40,43.59,3.11
2,beijing_grid_002,115.0,39.2,2017-01-01 00:00:00,-5.70,963.14,71.80,0.97,2.75
3,beijing_grid_003,115.0,39.3,2017-01-01 00:00:00,-5.88,946.94,68.20,327.65,3.84
4,beijing_grid_004,115.0,39.4,2017-01-01 00:00:00,-5.34,928.80,58.81,317.85,6.14


In [20]:
bj_meter_grid_hist.loc[bj_meter_grid_hist['wind_direction'] >, :]

,stationName,longitude,latitude,utc_time,temperature,pressure,humidity,wind_direction,wind_speed/kph


In [18]:
sorted(bj_meter_grid_hist['wind_direction'].unique())[0]

0.0

In [30]:
meteros_stations = load_city_meter_grid('bj')

['temperature', 'pressure', 'humidity', 'wind_direction', 'wind_speed']
temperature
pressure
humidity
wind_direction
wind_speed


In [31]:
meteros_stations.head()

,beijing_grid_000_temperature,beijing_grid_001_temperature,beijing_grid_002_temperature,beijing_grid_003_temperature,beijing_grid_004_temperature,beijing_grid_005_temperature,beijing_grid_006_temperature,beijing_grid_007_temperature,beijing_grid_008_temperature,beijing_grid_009_temperature,...,beijing_grid_641_wind_speed,beijing_grid_642_wind_speed,beijing_grid_643_wind_speed,beijing_grid_644_wind_speed,beijing_grid_645_wind_speed,beijing_grid_646_wind_speed,beijing_grid_647_wind_speed,beijing_grid_648_wind_speed,beijing_grid_649_wind_speed,beijing_grid_650_wind_speed
utc_time,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-5.47,-5.53,-5.70,-5.88,-5.34,-4.81,-4.98,-5.49,-6.17,-7.17,...,4.30,3.62,2.35,1.69,2.53,3.28,4.11,4.64,5.42,5.73
2017-01-01 01:00:00,-2.69,-2.70,-2.70,-2.71,-2.41,-2.11,-2.57,-3.40,-4.34,-5.49,...,4.57,3.84,2.26,1.01,2.19,3.09,4.00,4.69,5.55,5.87
2017-01-01 02:00:00,0.08,0.14,0.30,0.46,0.52,0.59,-0.15,-1.30,-2.50,-3.82,...,4.83,4.11,2.30,0.36,2.12,3.04,3.96,4.78,5.70,6.02
2017-01-01 03:00:00,2.86,2.97,3.30,3.63,3.46,3.29,2.26,0.79,-0.67,-2.14,...,5.10,4.40,2.47,0.43,2.33,3.13,4.00,4.90,5.85,6.18
2017-01-01 04:00:00,3.74,3.83,4.08,4.34,4.10,3.85,2.77,1.26,-0.23,-1.67,...,5.11,4.56,2.98,0.81,1.14,2.24,3.35,4.53,5.76,6.18


### Deduplication


In [32]:
meteros_stations.loc[meteros_stations.duplicated(), :]

,beijing_grid_000_temperature,beijing_grid_001_temperature,beijing_grid_002_temperature,beijing_grid_003_temperature,beijing_grid_004_temperature,beijing_grid_005_temperature,beijing_grid_006_temperature,beijing_grid_007_temperature,beijing_grid_008_temperature,beijing_grid_009_temperature,...,beijing_grid_641_wind_speed,beijing_grid_642_wind_speed,beijing_grid_643_wind_speed,beijing_grid_644_wind_speed,beijing_grid_645_wind_speed,beijing_grid_646_wind_speed,beijing_grid_647_wind_speed,beijing_grid_648_wind_speed,beijing_grid_649_wind_speed,beijing_grid_650_wind_speed
utc_time,,,,,,,,,,,,,,,,,,,,,


There's no duplication in the grid meterology data

### Missing data

In [19]:
# load data
bj_meter = load_city_meter('bj')

temperature
pressure
humidity
wind_direction
wind_speed


In [55]:
stations = get_stations('bj', grid=False)
stations

,stationId,longitude,latitude
0,dongsi_aq,116.417,39.929
1,tiantan_aq,116.407,39.886
2,guanyuan_aq,116.339,39.929
3,wanshouxigong_aq,116.352,39.878
4,aotizhongxin_aq,116.397,39.982
5,nongzhanguan_aq,116.461,39.937
6,wanliu_aq,116.287,39.987
7,beibuxinqu_aq,116.174,40.090
8,zhiwuyuan_aq,116.207,40.002
9,fengtaihuayuan_aq,116.279,39.863


In [58]:
meter_preprocess('bj')

temperature
pressure
humidity
wind_direction
wind_speed
Fill missing data for temperature
There are 107 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 87 still missing in the data 

Fill missing data for pressure
There are 107 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 87 still missing in the data 

Fill missing data for humidity
There are 107 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 87 still missing in the data 

Fill missing data for wind_direction
There are 107 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 87 still missing in the data 

Fill missing data for wind_speed
There are 107 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 87 still missing in the data 



In [ ]:
bj_grid_meter_fill_na = grid_meter_preprocess('bj')

temperature
pressure
humidity
wind_direction
wind_speed
Fill missing data for temperature
There are 132 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes


In [8]:
grids = get_stations('bj', grid=True)
stations = get_stations('bj')

In [17]:
find_nearest_grid(stations, grids)

{'dongsi_aq': 'beijing_grid_303',
 'tiantan_aq': 'beijing_grid_303',
 'guanyuan_aq': 'beijing_grid_282',
 'wanshouxigong_aq': 'beijing_grid_303',
 'aotizhongxin_aq': 'beijing_grid_304',
 'nongzhanguan_aq': 'beijing_grid_324',
 'wanliu_aq': 'beijing_grid_283',
 'beibuxinqu_aq': 'beijing_grid_263',
 'zhiwuyuan_aq': 'beijing_grid_262',
 'fengtaihuayuan_aq': 'beijing_grid_282',
 'yungang_aq': 'beijing_grid_239',
 'gucheng_aq': 'beijing_grid_261',
 'fangshan_aq': 'beijing_grid_238',
 'daxing_aq': 'beijing_grid_301',
 'yizhuang_aq': 'beijing_grid_323',
 'tongzhou_aq': 'beijing_grid_366',
 'shunyi_aq': 'beijing_grid_368',
 'pingchang_aq': 'beijing_grid_264',
 'mentougou_aq': 'beijing_grid_240',
 'pinggu_aq': 'beijing_grid_452',
 'huairou_aq': 'beijing_grid_349',
 'miyun_aq': 'beijing_grid_392',
 'yanqin_aq': 'beijing_grid_225',
 'dingling_aq': 'beijing_grid_265',
 'badaling_aq': 'beijing_grid_224',
 'miyunshuiku_aq': 'beijing_grid_414',
 'donggaocun_aq': 'beijing_grid_452',
 'yongledian_aq': 

### London Air Quality Data

In [171]:
ld_aq = pd.read_csv(r'./Data/London/London_historical_aqi_forecast_stations_20180331.csv')
ld_aq_other = pd.read_csv(r'./Data/London/London_historical_aqi_other_stations_20180331.csv')
ld_aq_new = pd.read_csv(r'./Data/London/ld_aq_new.csv')

c:\users\zhich\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [113]:
bj_particles_aq = aq_city_preprocess('bj')

Fill missing data for PM2.5
There should be 11620 dates in total
After filling, there are 11620 dates in total
There are 10221 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 465 still missing in the data 

Fill missing data for PM10
There should be 11620 dates in total
After filling, there are 11620 dates in total
There are 11395 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 482 still missing in the data 

Fill missing data for NO2
There should be 11620 dates in total
After filling, there are 11620 dates in total
There are 9936 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 465 still missing in the data 

Fill missing data for CO
There should be 11620 dates in total
After filling, there are 11620 dates in total
There are 10324 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 1016 still missing in the data 

Fill missing data for O3
There should be 11620 dates in total


In [108]:
ld_particles_aq = aq_city_preprocess('ld')

c:\users\zhich\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2963: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Fill missing data for PM2.5
There should be 11633 dates in total
After filling, there are 11633 dates in total
There are 11633 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 0 still missing in the data 

Fill missing data for PM10
There should be 11633 dates in total
After filling, there are 11633 dates in total
There are 11633 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 0 still missing in the data 

Fill missing data for NO2
There should be 11633 dates in total
After filling, there are 11633 dates in total
There are 10439 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 0 still missing in the data 



In [111]:
for particle, data in ld_particles_aq.items():
    print(data.shape)

(11633, 24)
(11633, 24)
(11633, 24)


In [112]:
for particle, data in ld_particles_aq.items():
    start_time = data.index.min()
    end_time = data.index.max()
    print('min date:', data.index.min())
    print('max date:', data.index.max())
    
    all_time = pd.date_range(start_time, end_time, freq='H')
    print('There should be {} dates in total'.format(len(all_time)))
    print('dates in the data', data.shape[0])

min date: 2017-01-01 00:00:00
max date: 2018-04-30 16:00:00
There should be 11633 dates in total
dates in the data 11633
min date: 2017-01-01 00:00:00
max date: 2018-04-30 16:00:00
There should be 11633 dates in total
dates in the data 11633
min date: 2017-01-01 00:00:00
max date: 2018-04-30 16:00:00
There should be 11633 dates in total
dates in the data 11633


### London Meo Data

In [4]:
ld_grid_meo = pd.read_csv(r'./Data/London/London_historical_meo_grid.csv')
ld_grid_meo_new = pd.read_csv(r'./Data/London/ld_')
ld_grid_stations = get_stations('ld', grid=True)

In [67]:
ld_grid_meo_new.head()

,stationId,utc_time,weather,temperature,pressure,humidity,wind_direction,wind_speed
0,london_grid_000,2018-03-31 07:00:00,CLOUDY,6.14,995.3492,85.0,304.13,21.98
1,london_grid_001,2018-03-31 07:00:00,CLOUDY,5.43,994.1751,88.0,303.32,18.90
2,london_grid_002,2018-03-31 07:00:00,CLOUDY,4.73,993.0009,91.0,302.20,15.82
3,london_grid_003,2018-03-31 07:00:00,RAIN,4.25,990.1415,93.0,303.02,14.06
4,london_grid_004,2018-03-31 07:00:00,RAIN,3.99,985.5967,95.0,306.29,13.63


In [66]:
ld_grid_meo.tail()

,stationName,longitude,latitude,utc_time,temperature,pressure,humidity,wind_direction,wind_speed/kph
9303961,london_grid_856,2.0,52.1,2018-03-27 05:00:00,7.06,1011.21,85.77,189.56,27.38
9303962,london_grid_857,2.0,52.2,2018-03-27 05:00:00,7.03,1011.11,85.88,189.88,26.71
9303963,london_grid_858,2.0,52.3,2018-03-27 05:00:00,6.91,1011.01,85.91,189.05,25.75
9303964,london_grid_859,2.0,52.4,2018-03-27 05:00:00,6.79,1010.92,85.94,188.15,24.79
9303965,london_grid_860,2.0,52.5,2018-03-27 05:00:00,6.74,1010.89,85.95,187.84,24.47


In [58]:
grid_meteros_stations = load_city_meter_grid('ld')

temperature
pressure
humidity
wind_direction
wind_speed


In [63]:
grid_meteros_stations = meter_preprocess('ld')

temperature
pressure
humidity
wind_direction
wind_speed
Fill missing data for temperature
There should be 11636 dates in total
After filling, there are 11636 dates in total
There are 106 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 87 still missing in the data 

Fill missing data for pressure
There should be 11636 dates in total
After filling, there are 11636 dates in total
There are 106 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 87 still missing in the data 

Fill missing data for humidity
There should be 11636 dates in total
After filling, there are 11636 dates in total
There are 106 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 87 still missing in the data 

Fill missing data for wind_direction
There should be 11636 dates in total
After filling, there are 11636 dates in total
There are 106 missing in the data
STMVL: Preparing
STMVL: Start with 8 processes
There are 87 still missing in th

In [65]:
ld_ws = grid_meteros_stations['wind_speed']
ld_ws.loc[ld_ws.isnull().all(axis=1), :]

,BX9_wind_speed,BX1_wind_speed,BL0_wind_speed,CD9_wind_speed,CD1_wind_speed,CT2_wind_speed,CT3_wind_speed,CR8_wind_speed,GN0_wind_speed,GR4_wind_speed,...,HV1_wind_speed,LH0_wind_speed,KC1_wind_speed,KF1_wind_speed,LW2_wind_speed,RB7_wind_speed,TD5_wind_speed,ST5_wind_speed,TH4_wind_speed,MY7_wind_speed
2018-03-27 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-27 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-27 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-27 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-27 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-27 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-27 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-27 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-27 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-27 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
for meo, data in grid_meteros_stations.items():
    start_time = data.index.min()
    end_time = data.index.max()
    print('min date:', data.index.min())
    print('max date:', data.index.max())
    
    all_time = pd.date_range(start_time, end_time, freq='H')
    print('There should be {} dates in total'.format(len(all_time)))
    print('dates in the data', data.shape[0])

min date: 2017-01-01 00:00:00
max date: 2018-04-30 19:00:00
There should be 11636 dates in total
dates in the data 11636
min date: 2017-01-01 00:00:00
max date: 2018-04-30 19:00:00
There should be 11636 dates in total
dates in the data 11636
min date: 2017-01-01 00:00:00
max date: 2018-04-30 19:00:00
There should be 11636 dates in total
dates in the data 11636
min date: 2017-01-01 00:00:00
max date: 2018-04-30 19:00:00
There should be 11636 dates in total
dates in the data 11636
min date: 2017-01-01 00:00:00
max date: 2018-04-30 19:00:00
There should be 11636 dates in total
dates in the data 11636
